In [200]:
import requests
import re
import json
import copy
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
print "Done"

Done


In [201]:
# define all functions
def distance(x, y):
    dist = float(0)
    for key in x.keys():
        dist += np.square(x[key]-y[key])
    return np.sqrt(dist)

def find_label(hero_name, hero_dict, centroids):
    dist1 = distance(hero_dict[hero_name], centroids[1])
    dist2 = distance(hero_dict[hero_name], centroids[2])
    dist3 = distance(hero_dict[hero_name], centroids[3])
    if (dist1 < dist2) and (dist1 < dist3):
        return 1, dist1
    elif (dist2 < dist1) and (dist2 < dist3):
        return 2, dist2
    else:
        return 3, dist3

def find_centroid(num, hero_dict):
    count = count_per_cluster()
    count = count[num-1]
    attributes = hero_dict['Zeus']
    vals = { attribute:0 for attribute in attributes}
    for entry in hero_data_by_label:
        if hero_data_by_label[entry] == num:
            for attribute in attributes:
                vals[attribute]+=hero_dict[entry][attribute]
    for val in vals:
        vals[val] = vals[val]/float(count)
    return vals

def count_per_cluster():
    count_label_1 = 0
    count_label_2 = 0
    count_label_3 = 0
    for entry in hero_data_by_label:
        if hero_data_by_label[entry] == 1:
            count_label_1+=1
        elif hero_data_by_label[entry] == 2:
            count_label_2+=1
        elif hero_data_by_label[entry] == 3:
            count_label_3+=1
    return count_label_1, count_label_2, count_label_3

def knn(hero_dict, centroids):
    old_centroids = copy.deepcopy(centroids)
    iterations = 0
    while True:
        total_distance = 0
        for hero in hero_data_by_name:
            label, distance = find_label(hero, hero_dict, centroids)
            hero_data_by_label[hero] = label
            total_distance+=distance
#             print distance, hero_data_by_name[hero]['ID']
        for i in range(1,4):
            centroids[i] = find_centroid(i, hero_dict)
        average_distance = (total_distance/111)
#         print "Average distance: %s" % average_distance
        iterations+=1
        if centroids == old_centroids:
            break
        old_centroids = copy.deepcopy(centroids)
    print "iterations: %s" % iterations
    print "Average distance: %s" % average_distance
#     pprint(hero_data_by_label)
#     pprint(centroids)
    return count_per_cluster(), average_distance
print "Done"

Done


In [202]:
#define all data

r = requests.get('http://api.herostats.io/heroes/all')
r.status_code
r.json
hero_data = r.json()

# pprint(hero_data)

hero_data_by_name = {}
hero_data_by_label = {}
hero_data_by_type_attributes_only = {}
hero_type_attribute = ['BaseInt', 'BaseStr', 'BaseAgi']

for hero in hero_data:
    temp_hero = {}
    temp_hero = hero_data[hero]
    temp_hero_name = temp_hero['Name']
    temp_hero.pop(temp_hero_name, None)
    temp_hero.pop('Patch', None)
    hero_data_by_name[temp_hero_name] = temp_hero

# pprint(hero_data_by_name)

for entry in hero_data_by_name:
    attr_dict = {}
    for attribute in hero_data_by_name[entry]:
        if attribute in hero_type_attribute:
            attr_dict[attribute] = hero_data_by_name[entry][attribute]
        hero_data_by_type_attributes_only[entry] = attr_dict

# pprint(hero_data_by_type_attributes_only)

def random_centroids():
    RandomInit = []
    while len(RandomInit)<3:
        num = np.random.randint(1,111)
        if num not in RandomInit:
            RandomInit.append(num)
    centroids = {
        1: hero_data_by_type_attributes_only.values()[RandomInit[0]],
        2: hero_data_by_type_attributes_only.values()[RandomInit[1]],
        3: hero_data_by_type_attributes_only.values()[RandomInit[2]]
    }
#     centroids = {
#         1: hero_data_by_type_attributes_only['Batrider'],
#         2: hero_data_by_type_attributes_only['Naga Siren'],
#         3: hero_data_by_type_attributes_only['Slark']
#     }
    return centroids

centroids = random_centroids()

#creates dictionary of heroes by clustering them in appropriate clusters
# for entry in hero_data_by_name:
#     hero_data_by_label[entry] = find_label(entry, hero_data_by_type_attributes_only, centroids)

# pprint(hero_data_by_label)
print "Done"

Done


In [203]:
# string = "Ancient Apparition	Ancient Apparition	295	49.0	522	1.80	578	325	600	18	20	25	INTBane	Bane	310	58.0	598	3.52	724	286	400	22	22	22	INTBatrider	Batrider	290	40.0	598	2.10	673	312	375	22	15	24	INTChen	Chen	300	50.5	560	1.10	596	273	600	23	15	21	INTCrystal Maiden	Crystal Maiden	280	41.0	484	1.24	520	273	600	16	16	16	INTDark Seer	Dark Seer	300	63.0	598	6.68	837	377	128 (Melee)	22	12	25	INTDazzle	Dazzle	305	50.0	484	1.94	540	351	550	16	21	27	INTDeath Prophet	Death Prophet	310	50.0	541	2.96	637	260	600	19	14	23	INTDisruptor	Disruptor	300	51.0	541	1.10	576	286	600	19	15	22	INTEnchantress	Enchantress	335	52.0	484	0.66	503	208	550	16	19	16	INTEnigma	Enigma	300	45.0	503	3.96	622	260	500	17	14	20	INTInvoker	Invoker	280	38.0	541	1.80	599	286	600	17	20	22	INTJakiro	Jakiro	290	57.0	636	2.40	727	364	400	25	10	28	INTKeeper of the Light	Keeper of the Light	315	46.0	446	1.10	475	286	600	14	15	22	INTLeshrac	Leshrac	320	43.0	484	3.22	577	338	600	16	23	26	INTLich	Lich	315	46.5	522	1.10	556	234	550	18	15	18	INTLina	Lina	295	49.0	522	1.24	560	351	670	18	16	27	INTLion	Lion	290	52.0	484	1.10	515	286	600	16	15	22	INTNature's Prophet	Nature's Prophet	295	52.0	541	3.52	655	273	600	19	18	25	INTNecrophos	Necrophos	290	47.0	484	3.10	574	286	550	16	15	22	INTOgre Magi	Ogre Magi	295	61.0	617	7.96	911	221	128 (Melee)	23	14	17	INTOracle	Oracle	305	48.0	522	2.10	587	299	620	18	15	23	INTOutworld Devourer	Outworld Devourer	315	53.5	541	5.36	714	338	450	19	24	26	INTPuck	Puck	295	52.5	465	2.08	523	312	550	15	22	24	INTPugna	Pugna	330	49.0	503	1.24	540	338	600	17	16	26	INTQueen of Pain	Queen of Pain	295	53.0	484	1.52	528	312	550	16	18	24	INTRubick	Rubick	290	49.0	541	0.96	572	351	600	19	14	27	INTShadow Demon	Shadow Demon	295	52.0	503	2.52	579	338	500	17	18	23	INTShadow Shaman	Shadow Shaman	285	50.5	541	1.24	581	273	500	21	16	21	INTSilencer	Silencer	295	50.0	503	1.24	540	351	600	17	16	27	INTSkywrath Mage	Skywrath Mage	325	44.0	541	-0.18	535	351	600	19	13	27	INTStorm Spirit	Storm Spirit	290	50.0	541	5.08	705	299	480	19	22	23	INTTechies	Techies	270	30.0	503	6.96	713	286	700	17	14	22	INTTinker	Tinker	305	52.0	503	3.82	618	351	550	17	13	27	INTVisage	Visage	285	50.0	598	-0.46	581	312	600	22	11	24	INTWarlock	Warlock	295	51.0	522	2.40	597	312	600	22	10	24	INTWindrunner	Windrunner	295	50.0	465	1.38	503	286	600	15	17	22	INTWinter Wyvern	Winter Wyvern	285	41.5	636	1.24	683	325	425	24	16	25	INTWitch Doctor	Witch Doctor	305	56.0	484	0.82	507	312	600	16	13	24	INTZeus	Zeus	295	45.0	541	1.54	590	260	350	19	11	20	INT"
# string = "Anti-Mage	Anti-Mage	315	51.0	560	2.08	629	195	128 (Melee)	22	22	15	AGIArc Warden	Arc Warden	295	45.0	636	0.10	639	312	625	24	15	24	AGIBloodseeker	Bloodseeker	290	56.0	617	3.36	741	234	128 (Melee)	23	24	18	AGIBounty Hunter	Bounty Hunter	315	52.0	503	5.94	682	247	128 (Melee)	17	21	19	AGIBroodmother	Broodmother	295	47.0	503	2.52	579	234	128 (Melee)	17	18	18	AGIClinkz	Clinkz	300	40.0	465	2.08	523	208	630	15	22	16	AGIDrow Ranger	Drow Ranger	300	49.5	503	0.64	522	195	625	17	26	15	AGIEmber Spirit	Ember Spirit	310	54.0	541	1.08	576	260	128 (Melee)	19	22	20	AGIFaceless Void	Faceless Void	300	61.0	617	3.94	762	195	128 (Melee)	23	23	15	AGIGyrocopter	Gyrocopter	315	46.0	522	4.36	658	299	365	18	24	19	AGIJuggernaut	Juggernaut	305	46.0	560	2.80	654	182	128 (Melee)	20	26	14	AGILone Druid	Lone Druid	325	48.0	503	3.36	604	169	550	17	24	13	AGILuna	Luna	330	51.0	465	4.08	578	208	330	15	18	16	AGIMedusa	Medusa	290	47.0	446	1.80	494	247	600	14	20	19	AGIMeepo	Meepo	315	42.0	617	4.22	773	260	128 (Melee)	23	23	20	AGIMirana	Mirana	300	43.5	503	1.80	557	221	630	17	20	17	AGIMorphling	Morphling	285	41.5	541	0.66	562	221	350	19	24	17	AGINaga Siren	Naga Siren	320	45.0	579	5.94	785	234	128 (Melee)	21	21	21	AGINyx Assassin	Nyx Assassin	300	51.0	522	3.66	636	234	128 (Melee)	18	19	18	AGIPhantom Assassin	Phantom Assassin	310	47.0	560	4.22	701	169	128 (Melee)	20	23	15	AGIPhantom Lancer	Phantom Lancer	290	56.0	522	3.22	622	273	128 (Melee)	18	29	18	AGIRazor	Razor	295	46.0	579	2.08	651	247	475	21	22	19	AGIRiki	Riki	290	40.0	503	4.76	646	182	128 (Melee)	17	34	14	AGIShadow Fiend	Shadow Fiend	305	38.0	465	0.80	487	234	500	15	20	18	AGISlark	Slark	305	58.0	579	1.94	646	208	128 (Melee)	21	21	16	AGISniper	Sniper	290	39.0	484	1.94	540	195	550	16	21	15	AGISpectre	Spectre	295	48.0	541	3.22	645	208	128 (Melee)	19	23	16	AGITemplar Assassin	Templar Assassin	305	56.0	522	4.22	654	260	140	18	23	20	AGITerrorblade	Terrorblade	315	51.0	465	7.08	662	247	128 (Melee)	15	22	19	AGITroll Warlord	Troll Warlord	305	47.0	503	1.94	561	169	500	17	21	13	AGIUrsa	Ursa	310	47.0	617	5.52	821	208	128 (Melee)	23	18	16	AGIVengeful Spirit	Vengeful Spirit	300	46.0	484	3.78	593	195	400	18	27	15	AGIVenomancer	Venomancer	285	42.0	522	3.08	618	234	450	18	22	18	AGIViper	Viper	285	45.0	560	1.94	625	195	575	20	21	15	AGIWeaver	Weaver	290	58.0	465	0.96	491	195	425	15	14	15	AGI"
# string = re.sub("\t", '', string)
# string = re.sub("\(Melee\)", '', string)
# string = re.sub("[+-]?\d+(?:\.\d+)*.AGI", '"', string)
# string = re.sub("[+-]?\d+(?:\.\d+)?.", '"\n', string)

# print string
# for i  in range(len(StrHeroes)):
#     StrHeroes[i] = StrHeroes[i][:len(StrHeroes[i])/2]

# print StrHeroes

In [204]:
IntHeroes = [ "Ancient Apparition", "Bane", "Batrider", "Chen", "Crystal Maiden", "Dark Seer", "Dazzle",
    "Death Prophet", "Disruptor", "Enchantress", "Enigma", "Invoker", "Jakiro", "Keeper of the Light", "Leshrac", 
    "Lich", "Lina", "Lion", "Nature's Prophet", "Necrophos", "Ogre Magi", "Oracle", "Outworld Devourer", "Puck",
    "Pugna", "Queen of Pain", "Rubick", "Shadow Demon", "Shadow Shaman", "Silencer",    "Skywrath Mage",
    "Storm Spirit", "Techies", "Tinker", "Visage", "Warlock", "Windrunner", "Winter Wyvern", "Witch Doctor", "Zeus"
]
StrHeroes = ['Abaddon', 'Alchemist', 'Axe', 'Beastmaster', 'Brewmaster', 'Bristleback', 'Centaur Warrunner',
    'Chaos Knight', 'Clockwerk', 'Doom', 'Dragon Knight', 'Earth Spirit', 'Earthshaker', 'Elder Titan', 'Huskar',
     'Io', 'Kunkka', 'Legion Commander', 'Lifestealer', 'Lycan', 'Magnus', 'Night Stalker', 'Omniknight', 'Phoenix',
     'Pudge', 'Sand King', 'Slardar', 'Spirit Breaker', 'Sven', 'Tidehunter', 'Timbersaw', 'Tiny', 'Treant Protector',
     'Tusk', 'Undying', 'Wraith King'
]
AgiHeroes = ['Anti-Mage', 'Arc Warden', 'Bloodseeker', 'Bounty Hunter', 'Broodmother', 'Clinkz', 'Drow Ranger',
    'Ember Spirit', 'Faceless Void', 'Gyrocopter', 'Juggernaut', 'Lone Druid', 'Luna', 'Medusa', 'Meepo', 'Mirana',
    'Morphling', 'Naga Siren', 'Nyx Assassin', 'Phantom Assassin', 'Phantom Lancer', 'Razor', 'Riki', 'Shadow Fiend',
    'Slark', 'Sniper', 'Spectre', 'Templar Assassin', 'Terrorblade', 'Troll Warlord', 'Ursa', 'Vengeful Spirit',
    'Venomancer', 'Viper', 'Weaver'
]
print "Done"

Done


In [205]:
# correct = 0
# correctAgi = 0
# correctStr = 0
# correctInt = 0

# for entry in hero_data_by_label:
#     if hero_data_by_label[entry] == 1:
#         if entry in IntHeroes:
#             correctInt+=1
#             correct+=1
#     elif hero_data_by_label[entry] == 2:
#         if entry in AgiHeroes:
#             correctAgi+=1
#             correct+=1
#     elif hero_data_by_label[entry] == 3:
#         if entry in StrHeroes:
#             correctStr+=1
#             correct+=1
# print correct, correctInt, correctAgi, correctStr
# print count_per_cluster()
def checkcorrectness():
    cluster_lengths, average_distance = knn(hero_data_by_type_attributes_only, random_centroids())
    PredictedOne = []
    PredictedTwo = []
    PredictedThree = []
    for entry in hero_data_by_label:
        if hero_data_by_label[entry] == 1:
            PredictedOne.append(entry)
        elif hero_data_by_label[entry] == 2:
            PredictedTwo.append(entry)
        else:
            PredictedThree.append(entry)
    Internsection1len1 = len(list(set(PredictedOne) & set(IntHeroes)))
    Internsection1len2 = len(list(set(PredictedTwo) & set(IntHeroes))) 
    Internsection1len3 = len(list(set(PredictedThree) & set(IntHeroes))) 
    Internsection2len1 = len(list(set(PredictedOne) & set(StrHeroes)))
    Internsection2len2 = len(list(set(PredictedTwo) & set(StrHeroes)))
    Internsection2len3 = len(list(set(PredictedThree) & set(StrHeroes)))
    Internsection3len1 = len(list(set(PredictedOne) & set(AgiHeroes)))
    Internsection3len2 = len(list(set(PredictedTwo) & set(AgiHeroes)))
    Internsection3len3 = len(list(set(PredictedThree) & set(AgiHeroes)))
    if (Internsection1len1>Internsection1len2) and (Internsection1len1>Internsection1len3):
        len1 = Internsection1len1
    elif (Internsection1len2>Internsection1len1) and (Internsection1len2>Internsection1len3):
        len1 = Internsection1len2
    else:
        len1 = Internsection1len3
    if (Internsection2len1>Internsection2len2) and (Internsection2len1>Internsection2len3):
        len2 = Internsection2len1
    elif (Internsection1len2>Internsection2len1) and (Internsection2len2>Internsection2len3):
        len2 = Internsection2len2
    else:
        len2 = Internsection2len3
    if (Internsection3len1>Internsection3len2) and (Internsection3len1>Internsection3len3):
        len3 = Internsection3len1
    elif (Internsection3len2>Internsection3len1) and (Internsection3len2>Internsection3len3):
        len3 = Internsection3len2
    else:
        len3 = Internsection3len3
#     len1 = len(list(set(PredictedOne) & set(IntHeroes)))
#     if len1<len(list(set(PredictedTwo) & set(IntHeroes))): len1 = len(list(set(PredictedTwo) & set(IntHeroes))) 
#     if len1<len(list(set(PredictedThree) & set(IntHeroes))): len1 = len(list(set(PredictedThree) & set(IntHeroes)))
#     len2 = len(list(set(PredictedOne) & set(AgiHeroes)))
#     if len2<len(list(set(PredictedTwo) & set(AgiHeroes))): len2 = len(list(set(PredictedTwo) & set(AgiHeroes)))
#     if len2<len(list(set(PredictedThree) & set(AgiHeroes))): len2 = len(list(set(PredictedThree) & set(AgiHeroes)))
#     len3 = len(list(set(PredictedOne) & set(StrHeroes)))
#     if len3<len(list(set(PredictedTwo) & set(StrHeroes))): len3 = len(list(set(PredictedTwo) & set(StrHeroes)))
#     if len3<len(list(set(PredictedThree) & set(StrHeroes))): len3 = len(list(set(PredictedThree) & set(StrHeroes)))
    print "Cluster 1: %s, Cluster 2: %s, Cluster3: %s" %(cluster_lengths[0], cluster_lengths[1], cluster_lengths[2])
    print "MaxIntersection 1: %s, MaxIntersection 2: %s, MaxIntersection 3: %s" %(len1, len2, len3)
    print "OneInt: %s, OneStr: %s, OneAgi: %s" % (Internsection1len1, Internsection2len1, Internsection3len1)
    print "TwoInt: %s, TwoStr: %s, TwoAgi: %s" % (Internsection1len2, Internsection2len2, Internsection3len2)
    print "ThreeInt: %s, ThreeStr: %s, ThreeAgi: %s" % (Internsection1len3, Internsection2len3, Internsection3len3)
    return average_distance
for i in xrange(1):
    checkcorrectness()
print "Done"

iterations: 5
Average distance: 4.91114798247
Cluster 1: 48, Cluster 2: 30, Cluster3: 33
MaxIntersection 1: 26, MaxIntersection 2: 20, MaxIntersection 3: 26
OneInt: 26, OneStr: 20, OneAgi: 2
TwoInt: 13, TwoStr: 10, TwoAgi: 7
ThreeInt: 1, ThreeStr: 6, ThreeAgi: 26
Done


In [206]:
# names = []
# labels = []
# hero_attribute_val = []
# for hero in hero_data_by_name:
#     names.append(hero)
#     labels.append(hero_data_by_label[hero])
#     if hero_data_by_label[hero] == 1:
#         hero_attribute_val.append(hero_data_by_type_attributes_only[hero]['BaseStr'])
#     if hero_data_by_label[hero] == 2:
#         hero_attribute_val.append(hero_data_by_type_attributes_only[hero]['BaseAgi'])
#     if hero_data_by_label[hero] == 3:
#         hero_attribute_val.append(hero_data_by_type_attributes_only[hero]['BaseInt'])
# plt.scatter(hero_attribute_val[10:20], names[10:20], c=labels[10:20])
# plt.show()

In [207]:
average_distance_array = []
for i in xrange(50):
    average_distance_array.append(checkcorrectness())
print min(average_distance_array)
print "Done"

iterations: 5
Average distance: 4.26512099902
Cluster 1: 34, Cluster 2: 39, Cluster3: 38
MaxIntersection 1: 33, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 4, OneStr: 28, OneAgi: 2
TwoInt: 3, TwoStr: 4, TwoAgi: 32
ThreeInt: 33, ThreeStr: 4, ThreeAgi: 1
iterations: 5
Average distance: 4.25944149479
Cluster 1: 34, Cluster 2: 34, Cluster3: 43
MaxIntersection 1: 29, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 4, OneStr: 28, OneAgi: 2
TwoInt: 29, TwoStr: 4, TwoAgi: 1
ThreeInt: 7, ThreeStr: 4, ThreeAgi: 32
iterations: 6
Average distance: 4.25944149479
Cluster 1: 43, Cluster 2: 34, Cluster3: 34
MaxIntersection 1: 29, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 7, OneStr: 4, OneAgi: 32
TwoInt: 29, TwoStr: 4, TwoAgi: 1
ThreeInt: 4, ThreeStr: 28, ThreeAgi: 2
iterations: 11
Average distance: 4.25944149479
Cluster 1: 43, Cluster 2: 34, Cluster3: 34
MaxIntersection 1: 29, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 7, OneStr: 4, OneAgi: 32
TwoInt: 29, TwoStr: 4, 

OneInt: 3, OneStr: 4, OneAgi: 32
TwoInt: 33, TwoStr: 4, TwoAgi: 1
ThreeInt: 4, ThreeStr: 28, ThreeAgi: 2
iterations: 13
Average distance: 4.26512099902
Cluster 1: 34, Cluster 2: 39, Cluster3: 38
MaxIntersection 1: 33, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 4, OneStr: 28, OneAgi: 2
TwoInt: 3, TwoStr: 4, TwoAgi: 32
ThreeInt: 33, ThreeStr: 4, ThreeAgi: 1
iterations: 5
Average distance: 4.26512099902
Cluster 1: 39, Cluster 2: 38, Cluster3: 34
MaxIntersection 1: 33, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 3, OneStr: 4, OneAgi: 32
TwoInt: 33, TwoStr: 4, TwoAgi: 1
ThreeInt: 4, ThreeStr: 28, ThreeAgi: 2
iterations: 18
Average distance: 4.25944149479
Cluster 1: 34, Cluster 2: 43, Cluster3: 34
MaxIntersection 1: 29, MaxIntersection 2: 28, MaxIntersection 3: 32
OneInt: 29, OneStr: 4, OneAgi: 1
TwoInt: 7, TwoStr: 4, TwoAgi: 32
ThreeInt: 4, ThreeStr: 28, ThreeAgi: 2
iterations: 18
Average distance: 4.26512099902
Cluster 1: 38, Cluster 2: 39, Cluster3: 34
MaxIntersection 1